# BMG Labtech CLARIOstar (Plus)

| Summary | Photo |
|---------|-------|
| <ul style="font-size:15px; line-height:1.6; margin-top:0;"> <li><a href="https://www.bmglabtech.com/en/clariostar-plus/" target="_blank"><b>OEM Link</b></a></li> <li><b>Communication Protocol / Hardware:</b> Serial (FTDI) / USB</li> <li><b>Communication Level:</b> Firmware</li> <li><b>Measurement Modes:</b> Absorbance, Luminescence, Fluorescence</li> <li><b>Optical Systems:</b> Dual LVF Monochromator (320–840 nm), Physical Filters (240–900 nm), UV/Vis Spectrometer (220–1000 nm)</li> <li><b>Plate Delivery:</b> Drawer</li> <li><b>Additional Features:</b> Temperature control, Shaking, Configurable scan modes</li> <li>VID:PID <code>0403:BB68</code></li> </ul> | <div style="width:320px; text-align:center;"> ![clariostar](img/bmg-labtech-clariostar-plus.png) <br><i>Figure: BMG Labtech CLARIOstar Plus</i> </div> |

---
## Setup Instructions (Physical)

The CLARIOstar and CLARIOstar Plus require a minimum of two cable connections to be operational:
1. Power cord (standard IEC C13)
2. USB cable (USB-B with security screws at CLARIOstar end; USB-A at control PC end)

Optional:
If you have a plate stacking unit to use with the CLARIOstar (Plus), an additional RS-232 port is available on the CLARIOstar (Plus).

---
## Setup Instructions (Programmatic)

To control the BMG Labtech CLARIOstar (Plus), create a `PlateReader` frontend instance that uses a `CLARIOstarBackend` as its backend.

For convenience, store the backend as a separate variable so you can access CLARIOstar-specific features directly.

Set `protocol_mode` below to `"execution"` when connected to real hardware, or `"simulation"` to run the notebook end-to-end without a device.

In [1]:
from pylabrobot import verbose

verbose(True)

In [2]:
protocol_mode = "execution" # "execution" or "simulation"

In [3]:
import logging, os
from datetime import datetime
from pylabrobot.io import LOG_LEVEL_IO
ap_identifier = "test_protocol_1"
rounded_min = (datetime.now().minute // 10) * 10
ts = datetime.now().replace(minute=rounded_min, second=0, microsecond=0).strftime('%Y-%m-%d_%H-%M')

os.makedirs("./_logs", exist_ok=True)
fh = logging.FileHandler(f"./_logs/{ts}_{ap_identifier}_{protocol_mode}.log", mode="a")
fh.setLevel(LOG_LEVEL_IO)
fh.setFormatter(logging.Formatter("%(asctime)s [%(levelname)s] %(name)s - %(message)s"))

for name, lvl in [("pylabrobot", LOG_LEVEL_IO), ("manager", logging.DEBUG), ("device", logging.DEBUG)]:
    lgr = logging.getLogger(name)
    lgr.setLevel(lvl)
    if not any(isinstance(h, logging.FileHandler) and h.baseFilename == fh.baseFilename for h in lgr.handlers):
        lgr.addHandler(fh)

logger_manager = logging.getLogger("manager")
logger_manager.info(f"START AUTOMATED PROTOCOL\naP identifier: {ap_identifier}\n\nUser: Camillo\n")

In [4]:
from pylabrobot.plate_reading import PlateReader

if protocol_mode == "execution":
    from pylabrobot.plate_reading.bmg_labtech import CLARIOstarBackend
    clariostar_backend = CLARIOstarBackend()

elif protocol_mode == "simulation":
    from pylabrobot.plate_reading.bmg_labtech import CLARIOstarSimulatorBackend
    clariostar_backend = CLARIOstarSimulatorBackend()

pr = PlateReader(
    name="CLARIOstar",
    backend=clariostar_backend,
    size_x=0.0,
    size_y=0.0,
    size_z=0.0
)

If you have multiple FTDI devices connected, pass a `device_id` to the backend to select the correct one:

```python
clariostar_backend = CLARIOstarBackend(device_id="FT1234AB")
```

In [5]:
await pr.setup()

2026-02-20 19:17:22,286 - pylabrobot.io.ftdi - INFO - Successfully opened FTDI device: 430-2621
2026-02-20 19:17:22,545 - pylabrobot - INFO - read complete response: 476 bytes, 0201dc0c1325002600000100060546010302020900090d094c6001001eb0160d000220000e00032700000f000300000011000300000012000200001f00090100ec1ad2043a00212e00090104600003001eb4442f017845585f53503a313d6f70656e2c323d46696c74657220312c333d46696c74657220320d0a45585f4c503a313d6f70656e2c323d46696c74657220312c333d46696c74657220320d0a4253503a313d6f70656e2c323d46696c74657220312c333d46696c74657220322c343d46696c74657220330d0a454d5f53503a313d6f70656e2c323d46696c74657220312c333d46696c74657220320d0a454d5f4c503a313d6f70656e2c323d46696c74657220312c333d46696c74657220320d0a454d5f534c49543a313d6f70656e2c323d536c697420302e382c333d536c697420312e322c343d536c697420322e302c353d536c697420332e302c373d4650207061722c383d4650207065722c363d4c554d206c656e73650d0a45585f534c49543a313d6f70656e2c323d536c697420302e362c333d536c697420312e302c343d536c697420312e3

```{note}
Expected behaviour: the machine performs its initialization routine (FTDI connection, baudrate configuration, firmware initialization, and EEPROM data request).
```

---
## Defining a Plate

Every measurement requires a plate to be assigned to the plate reader. The backend dynamically encodes the plate geometry (dimensions, well positions, and well count) into the binary protocol — so you can use **any** PLR plate definition.

Here we use a standard Corning 96-well flat-bottom plate as an example:

In [6]:
from pylabrobot.resources.corning import Cor_96_wellplate_360ul_Fb

plate = Cor_96_wellplate_360ul_Fb("test_plate")
pr.assign_child_resource(plate)

---
## Query Machine


### EEPROM Machine Configuration Retrieval

During `setup()`, the backend queries the CLARIOstar's EEPROM (command `0x05 0x07`, 264 bytes) and firmware info (command `0x05 0x09`, 32 bytes). These responses are parsed into a `CLARIOstarConfig` dataclass.

#### What's decoded so far

| Field | Source | Status |
|-------|--------|--------|
| Model name, monochromator range, filter slots | Machine type code (EEPROM bytes 2-3) | Confirmed |
| Firmware version | Firmware info bytes 6-7 (uint16 BE / 1000) | Confirmed |
| Build date / time | Firmware info bytes 8-27 (null-terminated ASCII) | Confirmed |
| has_absorbance, has_fluorescence, has_luminescence, has_alpha | EEPROM bytes 11-14 | Confirmed |
| Serial number | FTDI chip (not in EEPROM) | Confirmed |
| has_pump1, has_pump2, has_stacker | Unknown offset | Needs 2nd unit with different options |


In [7]:
# View the parsed machine configuration
import dataclasses

config = clariostar_backend.get_machine_config()
if config is not None:
    for field in dataclasses.fields(config):
        print(f"  {field.name:25s} {getattr(config, field.name)}")
else:
    print("No config available (EEPROM not read yet)")

  serial_number             430-2621
  firmware_version          1.35
  firmware_build_timestamp  Nov 20 2020 11:51:21
  model_name                CLARIOstar Plus
  machine_type_code         38
  has_absorbance            True
  has_fluorescence          True
  has_luminescence          True
  has_alpha_technology      True
  has_pump1                 False
  has_pump2                 False
  has_stacker               False
  monochromator_range       (220, 1000)
  num_filter_slots          11


In [8]:
# View lifetime usage counters (queries the instrument each time — not cached)
counters = await clariostar_backend.request_usage_counters()
for name, value in counters.items():
    print(f"  {name:25s} {value:>12,}")

2026-02-20 19:17:34,414 - pylabrobot - INFO - read complete response: 50 bytes, 0200320c210500260000001e023d0000078c000004e7000003d9000273e5000012fa0000000a0000000a0000000a0005c70d
2026-02-20 19:17:34,555 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000000000c00001120d
2026-02-20 19:17:34,556 - pylabrobot - INFO - status: {'valid': True, 'initialized': True, 'plate_detected': True, 'z_probed': True}


  flashes                      1,966,653
  testruns                         1,932
  wells                          125,500
  well_movements                  98,500
  active_time_s                  160,741
  shake_time_s                     4,858
  pump1_usage                         10
  pump2_usage                         10
  alpha_time                          10


### Current Machine Status

The CLARIOstar reports its state as a dictionary of boolean status flags. You can query them at any time to check what the machine is doing.

The available flags are:

| Flag | Meaning | Shorthand |
|------|---------|-----------
| `standby` | Machine is in standby mode | |
| `valid` | Status response is valid | |
| `busy` | Machine is currently executing a command | `request_busy()` |
| `running` | A measurement run is in progress | |
| `unread_data` | Measurement data is available to read | |
| `initialized` | Instrument has been initialized | `request_initialization_status()` |
| `lid_open` | Top-front lid is open (filter and pump access) | |
| `drawer_open` | The drawer is open | `request_drawer_open()` |
| `plate_detected` | A plate is detected in the drawer | `request_plate_detected()` |
| `z_probed` | Z-height probing has completed | |
| `active` | Machine is active | |
| `filter_cover_open` | The filter cover is open | |

Flags without a shorthand can be accessed via `request_machine_status()["flag_name"]`.

In [9]:
status = await clariostar_backend.request_machine_status()
for flag, value in status.items():
    print(f"  {flag:20s} {value}")

2026-02-20 19:17:34,601 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000000000c00001120d


  standby              False
  valid                True
  busy                 False
  running              False
  unread_data          False
  initialized          True
  lid_open             False
  drawer_open          False
  plate_detected       True
  z_probed             True
  active               False
  filter_cover_open    False


In [10]:
# Convenience queries for common checks
if await clariostar_backend.request_plate_detected():
    # verify that the resource model is correct and that the plate is in the expected position
    print("Plate is in the drawer")
else:
    print("No plate detected")

if await clariostar_backend.request_busy():
    # verify that the machine is available for commands, i.e. not busy running a measurement
    print("Machine is busy")
else:
    print("Machine is ready")

2026-02-20 19:17:54,654 - pylabrobot - WARNING - timed out reading response
2026-02-20 19:17:54,655 - pylabrobot - INFO - read complete response: 17 bytes, 0200180c0105002600000000000000000d
2026-02-20 19:17:54,660 - pylabrobot - WARNING - Truncated response (17/24 bytes), retrying after drain...
2026-02-20 19:17:55,701 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000000000c00001120d
2026-02-20 19:17:55,739 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000000000c00001120d


Plate is in the drawer
Machine is ready


---
## Usage: Drawer

The CLARIOstar loads and unloads plates via a motorized drawer.

In [11]:
# Open the drawer (plate out)
await pr.open()

2026-02-20 19:17:55,786 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012500260000030000000000000000c00001350d
2026-02-20 19:17:55,924 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012500260000030000000000000000c00001350d
2026-02-20 19:17:55,925 - pylabrobot - INFO - status: {'valid': True, 'busy': True, 'initialized': True, 'plate_detected': True, 'z_probed': True}
2026-02-20 19:17:56,063 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012500260000030000000000000000c00001350d
2026-02-20 19:17:56,200 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012500260000030000000000000000c00001350d
2026-02-20 19:17:56,339 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012500260000030000000000000000c00001350d
2026-02-20 19:17:56,477 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012500260000030000000000000000c00001350d
2026-02-20 19:17:56,616 - pylabrobot - INFO - read complete response: 24 bytes,

In [12]:
# Place your plate on the drawer (manually or via a robotic arm), then close it (plate in)
await pr.close()

2026-02-20 19:18:00,126 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012500210000030000000000000000c00001300d
2026-02-20 19:18:00,265 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012500210000030000000000000000c00001300d
2026-02-20 19:18:00,266 - pylabrobot - INFO - status: {'valid': True, 'busy': True, 'initialized': True, 'drawer_open': True}
2026-02-20 19:18:00,403 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012500210000030000000000000000c00001300d
2026-02-20 19:18:00,541 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012500210000030000000000000000c00001300d
2026-02-20 19:18:00,680 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012500210000030000000000000000c00001300d
2026-02-20 19:18:00,818 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012500210000030000000000000000c00001300d
2026-02-20 19:18:00,958 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012500210000

---
## Usage: Temperature

The CLARIOstar has a built-in incubator that heats from both below and above the microplate. The bottom plate tracks the setpoint; the top plate targets setpoint + 0.5 °C to prevent condensation on the plate seal. Two temperature sensors (one per heating plate) are reported via the firmware.

Every measurement response also includes an embedded temperature (the `"temperature"` key in result dicts). This is the bottom plate sensor sampled at measurement start — it matches `measure_temperature("bottom")` at thermal equilibrium.

### Measuring temperature (no heating)

Use `measure_temperature()` to activate the sensors and read the current temperature without turning on the incubator. This sends a "monitor only" command, waits briefly for the sensors to report, and returns a single float.

Select which sensor to read with the `sensor` parameter: `"bottom"` (below the plate, default), `"top"` (above the plate), or `"mean"` (average of both).

In [13]:
# Read current temperature (no heating) — defaults to bottom heating plate
temp = await clariostar_backend.measure_temperature()
print(f"Bottom plate: {temp:.1f} °C")

# Read the top heating plate
temp_top = await clariostar_backend.measure_temperature(sensor="top")
print(f"Top plate:    {temp_top:.1f} °C")

# Average of both
temp_mean = await clariostar_backend.measure_temperature(sensor="mean")
print(f"Mean:         {temp_mean:.1f} °C")

2026-02-20 19:18:06,963 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000000000c00001120d
2026-02-20 19:18:07,501 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000ef00f7c00002f80d
2026-02-20 19:18:07,541 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000000000c00001120d


Bottom plate: 23.9 °C


2026-02-20 19:18:08,081 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000ef00f7c00002f80d
2026-02-20 19:18:08,121 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000000000c00001120d


Top plate:    24.7 °C


2026-02-20 19:18:08,661 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010500260000000000000000ef00f7c00002f80d


Mean:         24.3 °C


### Incubation (heating to a target temperature)

Use `start_temperature_control()` to turn on the incubator. The target is in °C (range: ~3 °C above ambient to 45 °C, in 0.1 °C steps). Use `measure_temperature()` to poll the current reading while heating.

In [14]:
# import asyncio

# # Heat to 37 °C
# await clariostar_backend.start_temperature_control(37.0)

# # Monitor the temperature as it climbs
# for _ in range(5):
#     temp = await clariostar_backend.measure_temperature()
#     print(f"Bottom plate: {temp:.1f} °C")
#     await asyncio.sleep(2)

# # Switch off incubator when done
# await clariostar_backend.stop_temperature_control()

In [28]:
logger_manager.info(f"\n========================\nSTART absorbance measurement - column 1 - 600 nm - OD\n")
# Read only column 1 for faster testing
column_1_wells = [plate.get_well(f"{row}1") for row in "ABCDEFGH"]

results = await pr.read_absorbance(
    wavelength=600,
    wells=column_1_wells,
    use_new_return_type=True,
    report="OD",
    well_scan="point",
    wait=True,
    flashes_per_well=5,
    shake_speed_rpm=200,
    shake_duration_s=5,
)

print(f"OD600 of well A1: {results[0]['data'][0][0]}")
print(f"Temperature: {results[0]['temperature']} \u00b0C")
print(results)
logger_manager.info(f"\nEND absorbance measurement - column 1 - 600 nm - OD\n============================")

2026-02-20 19:23:34,285 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010506260000000000000000000000c00001180d
2026-02-20 19:23:34,288 - pylabrobot - INFO - Absorbance payload (117 bytes): 0431e82164059604682c521cfc0c088008008008008008008008000000000000000000000000000000000000000000000000000000000000000000000000008a02000010010005002004001e270f270f0101177000000064232826ca00000064000000000200000000000100000001000500010000
2026-02-20 19:23:34,376 - pylabrobot - INFO - read complete response: 53 bytes, 0200350c03250426000000002a9d0000002c010000000d00000001010000000000000001000000030001000000010000220001c00d
2026-02-20 19:23:34,378 - pylabrobot - INFO - Run command accepted: total_values=44, status=250426
2026-02-20 19:23:34,537 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400ef00f7c00003250d
2026-02-20 19:23:35,103 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400ef00f7c00003250d
2026-02-20 19:

OD600 of well A1: 0.07947963474752937
Temperature: 23.9 °C
[{'wavelength': 600, 'data': [[0.07947963474752937, None, None, None, None, None, None, None, None, None, None, None], [0.2682628395400806, None, None, None, None, None, None, None, None, None, None, None], [0.4146782085483953, None, None, None, None, None, None, None, None, None, None, None], [0.7185964416777828, None, None, None, None, None, None, None, None, None, None, None], [1.2922000099868742, None, None, None, None, None, None, None, None, None, None, None], [1.8636670634311323, None, None, None, None, None, None, None, None, None, None, None], [2.3837729154259883, None, None, None, None, None, None, None, None, None, None, None], [2.6910042013687656, None, None, None, None, None, None, None, None, None, None, None]], 'temperature': 23.9, 'time': 1771615421.7514012}]


In [30]:
logger_manager.info(f"\n========================\nSTART absorbance measurement - column 1 - 600 nm - OD\n")
# Read only column 1 for faster testing
column_1_wells = [plate.get_well(f"{row}1") for row in "ABCDEFGH"]

results = await pr.read_absorbance(
    wavelength=600,
    wells=column_1_wells,
    use_new_return_type=True,
    report="OD",
    well_scan="point",
    wait=True,
    flashes_per_well=5,
    shake_speed_rpm=200, # TODO: shaking doesn't elicit ANY REPONSE AT ALL
    shake_duration_s=20,
    settling_time_before_measurement=1
)

print(f"OD600 of well A1: {results[0]['data'][0][0]}")
print(f"Temperature: {results[0]['temperature']} \u00b0C")
print(results)
logger_manager.info(f"\nEND absorbance measurement - column 1 - 600 nm - OD\n============================")

2026-02-20 19:24:20,968 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010506260000000000000000000000c00001180d
2026-02-20 19:24:20,971 - pylabrobot - INFO - Absorbance payload (117 bytes): 0431e82164059604682c521cfc0c088008008008008008008008000000000000000000000000000000000000000000000000000000000000000000000000008a02000010010014002004001e270f270f0101177000000064232826ca00000064000100010200000000000100000001000500010000
2026-02-20 19:24:21,059 - pylabrobot - INFO - read complete response: 53 bytes, 0200350c03250426000000002a9d0000002c010000000e00000001010000000000000001000000030001000000010000200001bf0d
2026-02-20 19:24:21,060 - pylabrobot - INFO - Run command accepted: total_values=44, status=250426
2026-02-20 19:24:21,219 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400000000c000013f0d
2026-02-20 19:24:21,784 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c012504260000040100000400ef00f7c00003250d
2026-02-20 19:

OD600 of well A1: 0.07915784644167641
Temperature: 23.9 °C
[{'wavelength': 600, 'data': [[0.07915784644167641, None, None, None, None, None, None, None, None, None, None, None], [0.2694199550230282, None, None, None, None, None, None, None, None, None, None, None], [0.4146095698909128, None, None, None, None, None, None, None, None, None, None, None], [0.7151686210021131, None, None, None, None, None, None, None, None, None, None, None], [1.288719400066071, None, None, None, None, None, None, None, None, None, None, None], [1.874980183146392, None, None, None, None, None, None, None, None, None, None, None], [2.3800156820937475, None, None, None, None, None, None, None, None, None, None, None], [2.69108877698127, None, None, None, None, None, None, None, None, None, None, None]], 'temperature': 23.9, 'time': 1771615469.622816}]


In [33]:
logger_manager.info(f"\n========================\nSTART absorbance measurement - column 1 - 600 nm - OD\n")
# Read only column 1 for faster testing
column_1_wells = [plate.get_well(f"{row}1") for row in "ABCDEFGH"]

results = await pr.read_absorbance(
    wavelength=600,
    wells=column_1_wells,
    use_new_return_type=True,
    report="OD",
    well_scan="orbital",
    well_scan_width=3,
    wait=True,
    flashes_per_well=5,
)

print(f"OD600 of well A1: {results[0]['data'][0][0]}")
print(f"Temperature: {results[0]['temperature']} \u00b0C")
print(results)
logger_manager.info(f"\nEND absorbance measurement - column 1 - 600 nm - OD\n============================")

2026-02-20 19:26:05,097 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010506260000000000000000000000c00001180d
2026-02-20 19:26:05,099 - pylabrobot - INFO - Absorbance payload (142 bytes): 0431e82164059604682c521cfc0c08008008008008008008008008000000000000000000000000000000000000000000000000000000000000000000000000008a32000000000000000000000000000000000000000000000000000000000000270f270f020302ae000101177000000064232826ca00000064000000000200000000000100000001000500010000
2026-02-20 19:26:05,175 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c011500260000460000000000000000c00001680d
2026-02-20 19:26:05,176 - pylabrobot - INFO - Run command REJECTED: total_values=0, status=150026


RuntimeError: Absorbance run command rejected by firmware. Status bytes: 150026, raw response: 0200180c011500260000460000000000000000c00001680d

In [ ]:
await pr.backend.send_command(
    
)

In [24]:
import pandas as pd

In [25]:
pd.DataFrame(results[0]["data"])

,0,1,2,3,4,5,6,7,8,9,10,11
0,0.078524,0.085454,0.086330,0.089039,0.086251,0.085230,1.473111,0.092590,0.104519,0.087336,0.089895,0.087755
1,0.265098,0.081923,0.306306,0.103240,0.103426,0.087290,0.155340,0.097686,0.094143,0.088179,0.088486,0.085943
2,0.412187,0.078994,0.083877,0.553239,0.133421,0.090217,1.512223,0.109632,0.089884,0.092521,0.087841,0.085575
3,0.716049,0.210130,0.128313,0.083326,0.151950,0.085959,0.096924,0.094452,0.092733,0.096119,0.089502,0.090712
4,1.284616,0.095308,0.083895,0.077981,0.082294,0.084110,1.521995,0.099673,0.093174,0.105528,0.090226,0.109047
5,1.877471,0.203129,0.155820,0.138680,0.089278,0.089813,0.087349,0.097370,0.114418,0.091447,0.128278,0.166078
6,2.419606,0.117053,0.089342,0.090298,0.086983,0.094353,1.483524,0.095464,0.099526,0.087128,0.106981,0.100698
7,2.781950,0.085890,0.089196,0.091234,0.087595,0.084638,0.087728,0.088005,0.140530,0.096070,0.086621,0.109024


In [26]:
clariostar_backend._pending_measurement

In [16]:
# --- Shake test: well_scan="orbital" (uses 36-byte block with _SHAKE_TYPE_0026 encoding) ---
# Previous tests used default well_scan="point" which uses the compact format
# with Go-style shake encoding — shaking showed ZERO time overhead.
# This test uses well_scan="orbital" to exercise the 36-byte block path
# where shake bytes are at block positions 17, 23, 24-25.

import time
from pylabrobot.plate_reading.bmg_labtech.clariostar_backend import ShakerType

column_1_wells = [plate.get_well(f"{row}1") for row in "ABCDEFGH"]

def run_fmt(label, results, elapsed):
    a1 = results[0]["data"][0][0]
    h1 = results[0]["data"][7][0]
    temp = results[0]["temperature"]
    print(f"  {label:45s} {elapsed:6.1f}s  A1={a1:.4f}  H1={h1:.4f}  T={temp:.1f}")

print("=" * 90)
print("SHAKE DEBUG: well_scan='orbital', well_scan_width=3, column 1 (8 wells)")
print("  This path uses the 36-byte block with _SHAKE_TYPE_0026 encoding")
print("=" * 90)

# Baseline: orbital scan, no shake
print("\n-- Baseline: orbital scan, NO shake --")
for rep in range(2):
    t0 = time.time()
    results = await pr.read_absorbance(
        wavelength=600, wells=column_1_wells,
        well_scan="orbital", well_scan_width=3,
        use_new_return_type=True,
    )
    run_fmt(f"orbital_nosshake rep={rep+1}", results, time.time() - t0)

# Orbital shake 300 RPM x 5s (OEM-verified pcap combo)
print("\n-- Orbital shake 300 RPM x 5s (OEM-verified combo) --")
for rep in range(2):
    t0 = time.time()
    results = await pr.read_absorbance(
        wavelength=600, wells=column_1_wells,
        well_scan="orbital", well_scan_width=3,
        shake_type=ShakerType.ORBITAL, shake_speed_rpm=300, shake_duration_s=5,
        use_new_return_type=True,
    )
    run_fmt(f"orbital_shake_300rpm_5s rep={rep+1}", results, time.time() - t0)

# Orbital shake 200 RPM x 7s
print("\n-- Orbital shake 200 RPM x 7s --")
for rep in range(2):
    t0 = time.time()
    results = await pr.read_absorbance(
        wavelength=600, wells=column_1_wells,
        well_scan="orbital", well_scan_width=3,
        shake_type=ShakerType.ORBITAL, shake_speed_rpm=200, shake_duration_s=7,
        use_new_return_type=True,
    )
    run_fmt(f"orbital_shake_200rpm_7s rep={rep+1}", results, time.time() - t0)

# Linear shake 200 RPM x 7s
print("\n-- Linear shake 200 RPM x 7s --")
t0 = time.time()
results = await pr.read_absorbance(
    wavelength=600, wells=column_1_wells,
    well_scan="orbital", well_scan_width=3,
    shake_type=ShakerType.LINEAR, shake_speed_rpm=200, shake_duration_s=7,
    use_new_return_type=True,
)
run_fmt("linear_shake_200rpm_7s", results, time.time() - t0)

# Double orbital shake 200 RPM x 7s
print("\n-- Double orbital shake 200 RPM x 7s --")
t0 = time.time()
results = await pr.read_absorbance(
    wavelength=600, wells=column_1_wells,
    well_scan="orbital", well_scan_width=3,
    shake_type=ShakerType.DOUBLE_ORBITAL, shake_speed_rpm=200, shake_duration_s=7,
    use_new_return_type=True,
)
run_fmt("double_orbital_shake_200rpm_7s", results, time.time() - t0)

# Point scan with same shake for comparison (uses compact format)
print("\n-- COMPARISON: point scan + orbital shake 300 RPM x 5s (compact format) --")
t0 = time.time()
results = await pr.read_absorbance(
    wavelength=600, wells=column_1_wells,
    well_scan="point",
    shake_type=ShakerType.ORBITAL, shake_speed_rpm=300, shake_duration_s=5,
    use_new_return_type=True,
)
run_fmt("point_shake_300rpm_5s (compact fmt)", results, time.time() - t0)

print("\n" + "=" * 90)
print("If shaking works: orbital scan + shake should take ~5-7s LONGER than baseline")
print("If shaking broken: all times will be ~equal (firmware silently ignores)")
print("=" * 90)

2026-02-20 19:18:30,825 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c010506260000000000000000000000c00001180d
2026-02-20 19:18:30,827 - pylabrobot - INFO - Absorbance payload (142 bytes): 0431e82164059604682c521cfc0c08008008008008008008008008000000000000000000000000000000000000000000000000000000000000000000000000008a32000000000000000000000000000000000000000000000000000000000000270f270f020302ae000101177000000064232826ca00000064000000000200000000000100000001000500010000
2026-02-20 19:18:30,903 - pylabrobot - INFO - read complete response: 24 bytes, 0200180c011500260000460000000000000000c00001680d
2026-02-20 19:18:30,904 - pylabrobot - INFO - Run command REJECTED: total_values=0, status=150026


SHAKE DEBUG: well_scan='orbital', well_scan_width=3, column 1 (8 wells)
  This path uses the 36-byte block with _SHAKE_TYPE_0026 encoding

-- Baseline: orbital scan, NO shake --


RuntimeError: Absorbance run command rejected by firmware. Status bytes: 150026, raw response: 0200180c011500260000460000000000000000c00001680d

In [18]:
import pandas as pd

pd.DataFrame(results[0]['data'])


,0,1,2,3,4,5,6,7,8,9,10,11
0,0.478596,None,None,None,None,None,None,None,None,None,None,None
1,0.466912,None,None,None,None,None,None,None,None,None,None,None
2,0.500853,None,None,None,None,None,None,None,None,None,None,None
3,0.502610,None,None,None,None,None,None,None,None,None,None,None
4,0.523884,None,None,None,None,None,None,None,None,None,None,None
5,0.496807,None,None,None,None,None,None,None,None,None,None,None
6,0.458801,None,None,None,None,None,None,None,None,None,None,None
7,0.514417,None,None,None,None,None,None,None,None,None,None,None


---
## Usage: Measuring Absorbance

Absorbance measures how much light a sample absorbs at a given wavelength.
Results can be reported as optical density (OD) or percent transmittance.

**Transmittance** is the percentage of light that passes through the sample. A clear well lets ~100% through; a dense culture might let only 10% through.

**OD (optical density)** is the log-transformed version: `OD = log10(100 / T%)`. The log scale is useful because it is proportional to the concentration of the absorbing substance (Beer-Lambert law), whereas transmittance is not. Most protocols report OD.

| Transmittance | OD | Meaning |
|--------------:|---:|---------|
| 100% | 0.0 | No absorption (blank / empty well) |
| 10% | 1.0 | 90% of light absorbed |
| 1% | 2.0 | 99% of light absorbed |

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `wavelength` | `int` | *required* | Wavelength in nm (220–1000) |
| `report` | `"OD"`, `"transmittance"`, or `"raw"` | `"OD"` | Output format |
| `wavelengths` | `List[int]` | — | Multiple wavelengths (1–8), overrides `wavelength` |
| `flashes_per_well` | `int` | `5` | Xenon lamp flashes per well (1–200). OEM default. Values ≥ 5 give converged OD; each extra flash adds ~12 ms/well. |
| `pause_time_per_well` | `int` | `0` | Per-well positioning delay in deciseconds (0–10). Adds ~0.1 s/well/unit. Useful after shaking. |
| `settling_time_before_measurement` | `int` | `0` | Once-per-run post-shake settling delay in seconds. Adds ~1.1 s/unit. No effect on accuracy. |

For a detailed explanation of how the CLARIOstar converts raw detector counts to OD values, see [Reference: How Absorbance Calculation Works](#reference-how-absorbance-calculation-works) at the end of this notebook.

### Single-wavelength absorbance (OD)

In [19]:
import time, asyncio, statistics

# 16-well pattern across columns 1-3 (includes A1 and H1)
# Col 1: all 8 rows | Col 2: A,C,E,G (even rows) | Col 3: B,D,F,H (odd rows)
test_wells_16 = (
    [plate.get_well(f"{row}1") for row in "ABCDEFGH"]
    + [plate.get_well(f"{row}2") for row in "ACEG"]
    + [plate.get_well(f"{row}3") for row in "BDFH"]
)

# (row_index, col_index, label) for the 16 wells in the 8x12 grid
_W16 = (
    [(r, 0, f"{ch}1") for r, ch in enumerate("ABCDEFGH")]
    + [(r, 1, f"{ch}2") for r, ch in zip([0, 2, 4, 6], "ACEG")]
    + [(r, 2, f"{ch}3") for r, ch in zip([1, 3, 5, 7], "BDFH")]
)

def fmt16(results, label):
    """Print all 16 well OD values for a single measurement."""
    d, t = results[0]["data"], results[0]["temperature"]
    v = [f"{n}={d[r][c]:.4f}" if d[r][c] is not None else f"{n}=None"
         for r, c, n in _W16]
    print(f"  {label}  T={t:.1f}\u00b0C")
    print(f"    {' '.join(v[:8])}")
    print(f"    {' '.join(v[8:12])}")
    print(f"    {' '.join(v[12:])}")

def summarize(a1_vals, h1_vals, timings):
    """Print replicate summary statistics for A1, H1, and timing."""
    print(f"  -- A1: mean={statistics.mean(a1_vals):.4f}  std={statistics.stdev(a1_vals):.5f}")
    print(f"  -- H1: mean={statistics.mean(h1_vals):.4f}  std={statistics.stdev(h1_vals):.5f}")
    print(f"  -- Time: mean={statistics.mean(timings):.1f}s  std={statistics.stdev(timings):.2f}s\n")

print(f"Selected {len(test_wells_16)} wells: {', '.join(w.name for w in test_wells_16)}")

Selected 16 wells: test_plate_well_A1, test_plate_well_B1, test_plate_well_C1, test_plate_well_D1, test_plate_well_E1, test_plate_well_F1, test_plate_well_G1, test_plate_well_H1, test_plate_well_A2, test_plate_well_C2, test_plate_well_E2, test_plate_well_G2, test_plate_well_B3, test_plate_well_D3, test_plate_well_F3, test_plate_well_H3


---
### Parameter Exploration: pause_time_per_well, settling_time_before_measurement, shaking

Characterization of firmware measurement parameters and their factorial combinations.
Each test uses **16 wells** across columns 1-3 (staggered pattern: all 8 rows in col 1,
alternating rows in cols 2-3; A1 and H1 always included) at 600 nm.

**Test matrix:**
- **Cell 35** — `pause_time_per_well`: 2 values (0, 5) × 2 reps + vertical scan comparison
- **Cell 36** — `settling_time_before_measurement`: 2 values (0, 2) × 2 reps + vertical scan comparison
- **Cell 37** — **Shake factorial**: orbital/linear/double-orbital 200 RPM × 7 s combined with ptpw, stbm, and vertical

#### Previous Results Summary (from prior runs)

**`pause_time_per_well`**: Adds ~1.05 s/unit for 16 wells. No effect on OD accuracy. Linear timing.

**`settling_time_before_measurement`**: Adds ~1.06 s/unit. No effect on OD accuracy. Linear timing.

**`flashes_per_well`** (characterized, no longer tested): fpw≥5 converges to within ±0.001 OD.
fpw=1 gives +7% OD error — avoid. OEM default (5) is the sweet spot. ~12 ms/flash/well timing.

```{note}
**Transient firmware delay (~14 s)**: A one-time delay of ~14 s can occur once per session
at an unpredictable measurement position. This is a firmware-level lamp calibration or
thermal stabilization event. It does not affect measurement accuracy, only timing.
```

In [20]:
# --- Test 1: pause_time_per_well (trimmed) ---
# Per-well positioning delay (deciseconds). Reduced to 2 key values × 2 reps.
# Full sweep (0,1,5,10 × 3 reps) confirmed linear scaling in prior runs.

print("=" * 72)
print("pause_time_per_well: 2 replicates, ambient")
print("=" * 72)
for ptpw in [0, 5]:
    a1_v, h1_v, t_v = [], [], []
    for rep in range(2):
        t0 = time.time()
        results = await pr.read_absorbance(
            wavelength=600, wells=test_wells_16,
            pause_time_per_well=ptpw,
            use_new_return_type=True,
        )
        elapsed = time.time() - t0
        t_v.append(elapsed)
        a1_v.append(results[0]["data"][0][0])
        h1_v.append(results[0]["data"][7][0])
        fmt16(results, f"ptpw={ptpw:2d} rep={rep+1} {elapsed:5.1f}s")
    summarize(a1_v, h1_v, t_v)

print("=" * 72)
print("pause_time_per_well: vertical=True vs vertical=False")
print("=" * 72)
for ptpw in [0, 5]:
    for vert in [False, True]:
        t0 = time.time()
        results = await pr.read_absorbance(
            wavelength=600, wells=test_wells_16,
            pause_time_per_well=ptpw, vertical=vert,
            use_new_return_type=True,
        )
        elapsed = time.time() - t0
        fmt16(results, f"ptpw={ptpw:2d} vertical={str(vert):5s} {elapsed:5.1f}s")
    print()

2026-02-20 09:12:40,564 - pylabrobot - INFO - Abs parser: schema=0x29, wells=16, wl=1, groups=4 (1+3), payload=325 B, cal_pairs=4, sample[0]=33596, ref[0]=200799, c1_cal=(100000, 0), ref_cal=(200000, 0), temp=21.0
2026-02-20 09:12:40,567 - pylabrobot - INFO - Abs parser: schema=0x29, wells=16, wl=1, groups=4 (1+3), payload=325 B, cal_pairs=4, sample[0]=31057, ref[0]=198763, c1_cal=(100000, 0), ref_cal=(200000, 0), temp=21.0
2026-02-20 09:12:40,569 - pylabrobot - INFO - Abs parser: schema=0x29, wells=16, wl=1, groups=4 (1+3), payload=325 B, cal_pairs=4, sample[0]=31306, ref[0]=200637, c1_cal=(100000, 0), ref_cal=(200000, 0), temp=21.0
2026-02-20 09:12:40,570 - pylabrobot - INFO - Abs parser: schema=0x29, wells=16, wl=1, groups=4 (1+3), payload=325 B, cal_pairs=4, sample[0]=29920, ref[0]=197218, c1_cal=(100000, 0), ref_cal=(200000, 0), temp=21.0
2026-02-20 09:12:40,571 - pylabrobot - INFO - Abs parser: schema=0x29, wells=16, wl=1, groups=4 (1+3), payload=325 B, cal_pairs=4, sample[0]=334

pause_time_per_well: 2 replicates, ambient
  ptpw= 0 rep=1   0.0s  T=21.0°C
    A1=0.4754 B1=0.5037 C1=0.4716 D1=0.4904 E1=0.5035 F1=0.4945 G1=0.5211 H1=0.4932
    A2=0.4635 C2=0.5081 E2=0.5227 G2=0.5514
    B3=0.5320 D3=0.5236 F3=0.4778 H3=0.4942
  ptpw= 0 rep=2   0.0s  T=21.0°C
    A1=0.5051 B1=0.4962 C1=0.4800 D1=0.5029 E1=0.5114 F1=0.5105 G1=0.4230 H1=0.4971
    A2=0.5513 C2=0.5261 E2=0.6004 G2=0.4827
    B3=0.4999 D3=0.4955 F3=0.5145 H3=0.4855
  -- A1: mean=0.4903  std=0.02100
  -- H1: mean=0.4952  std=0.00271
  -- Time: mean=0.0s  std=0.00s

  ptpw= 5 rep=1   0.0s  T=21.0°C
    A1=0.5058 B1=0.4711 C1=0.5035 D1=0.4897 E1=0.4764 F1=0.4715 G1=0.4969 H1=0.5242
    A2=0.5289 C2=0.4962 E2=0.5258 G2=0.5404
    B3=0.5148 D3=0.4664 F3=0.5170 H3=0.5042
  ptpw= 5 rep=2   0.0s  T=21.0°C
    A1=0.5180 B1=0.4962 C1=0.5046 D1=0.5586 E1=0.5053 F1=0.4842 G1=0.4446 H1=0.5194
    A2=0.5067 C2=0.4727 E2=0.4833 G2=0.5101
    B3=0.5089 D3=0.5285 F3=0.5101 H3=0.5053
  -- A1: mean=0.5119  std=0.00863
  

In [21]:
# --- Test 2: settling_time_before_measurement (trimmed) ---
# Once-per-run delay before measurement starts (seconds).
# Reduced to 2 key values × 2 reps. Full sweep confirmed linear ~1.06 s/unit.

print("=" * 72)
print("settling_time_before_measurement: 2 replicates, ambient")
print("=" * 72)
for stbm in [0, 2]:
    a1_v, h1_v, t_v = [], [], []
    for rep in range(2):
        t0 = time.time()
        results = await pr.read_absorbance(
            wavelength=600, wells=test_wells_16,
            settling_time_before_measurement=stbm,
            use_new_return_type=True,
        )
        elapsed = time.time() - t0
        t_v.append(elapsed)
        a1_v.append(results[0]["data"][0][0])
        h1_v.append(results[0]["data"][7][0])
        fmt16(results, f"stbm={stbm:2d} rep={rep+1} {elapsed:5.1f}s")
    summarize(a1_v, h1_v, t_v)

print("=" * 72)
print("settling_time_before_measurement: vertical=True vs vertical=False")
print("=" * 72)
for stbm in [0, 2]:
    for vert in [False, True]:
        t0 = time.time()
        results = await pr.read_absorbance(
            wavelength=600, wells=test_wells_16,
            settling_time_before_measurement=stbm, vertical=vert,
            use_new_return_type=True,
        )
        elapsed = time.time() - t0
        fmt16(results, f"stbm={stbm:2d} vertical={str(vert):5s} {elapsed:5.1f}s")
    print()

2026-02-20 09:12:41,176 - pylabrobot - INFO - Abs parser: schema=0x29, wells=16, wl=1, groups=4 (1+3), payload=325 B, cal_pairs=4, sample[0]=29645, ref[0]=198788, c1_cal=(100000, 0), ref_cal=(200000, 0), temp=21.0
2026-02-20 09:12:41,178 - pylabrobot - INFO - Abs parser: schema=0x29, wells=16, wl=1, groups=4 (1+3), payload=325 B, cal_pairs=4, sample[0]=32701, ref[0]=195953, c1_cal=(100000, 0), ref_cal=(200000, 0), temp=21.0
2026-02-20 09:12:41,179 - pylabrobot - INFO - Abs parser: schema=0x29, wells=16, wl=1, groups=4 (1+3), payload=325 B, cal_pairs=4, sample[0]=32617, ref[0]=200040, c1_cal=(100000, 0), ref_cal=(200000, 0), temp=21.0
2026-02-20 09:12:41,180 - pylabrobot - INFO - Abs parser: schema=0x29, wells=16, wl=1, groups=4 (1+3), payload=325 B, cal_pairs=4, sample[0]=31681, ref[0]=200858, c1_cal=(100000, 0), ref_cal=(200000, 0), temp=21.0
2026-02-20 09:12:41,182 - pylabrobot - INFO - Abs parser: schema=0x29, wells=16, wl=1, groups=4 (1+3), payload=325 B, cal_pairs=4, sample[0]=337

settling_time_before_measurement: 2 replicates, ambient
  stbm= 0 rep=1   0.0s  T=21.0°C
    A1=0.5254 B1=0.5170 C1=0.5337 D1=0.5032 E1=0.4593 F1=0.4242 G1=0.5082 H1=0.5470
    A2=0.4930 C2=0.5406 E2=0.4795 G2=0.4755
    B3=0.5165 D3=0.4996 F3=0.4909 H3=0.5023
  stbm= 0 rep=2   0.0s  T=21.0°C
    A1=0.4766 B1=0.5584 C1=0.4994 D1=0.4527 E1=0.4843 F1=0.4721 G1=0.5155 H1=0.5385
    A2=0.4718 C2=0.4710 E2=0.5022 G2=0.5051
    B3=0.4840 D3=0.5410 F3=0.5108 H3=0.4983
  -- A1: mean=0.5010  std=0.03454
  -- H1: mean=0.5427  std=0.00602
  -- Time: mean=0.0s  std=0.00s

  stbm= 2 rep=1   0.0s  T=21.0°C
    A1=0.4866 B1=0.4598 C1=0.5245 D1=0.4761 E1=0.5034 F1=0.4843 G1=0.4563 H1=0.5077
    A2=0.5444 C2=0.4994 E2=0.5293 G2=0.4824
    B3=0.4677 D3=0.5006 F3=0.4990 H3=0.5221
  stbm= 2 rep=2   0.0s  T=21.0°C
    A1=0.5011 B1=0.5279 C1=0.4915 D1=0.4474 E1=0.4830 F1=0.5186 G1=0.5027 H1=0.5080
    A2=0.4899 C2=0.5119 E2=0.4866 G2=0.5004
    B3=0.4985 D3=0.5177 F3=0.5169 H3=0.5360
  -- A1: mean=0.4939  s

In [ ]:
# --- Test 3: Shake factorial combinations ---
# Orbital 200 RPM, 7 seconds — combined with ptpw, stbm, vertical, and shake type.
# Tests that shaking + measurement parameters work in combination and that
# the firmware accepts the composite payloads correctly.

from pylabrobot.plate_reading.bmg_labtech.clariostar_backend import ShakerType

SHAKE = dict(shake_type=ShakerType.ORBITAL, shake_speed_rpm=200, shake_duration_s=7)

print("=" * 72)
print("Shake factorial: orbital 200 RPM × 7 s")
print("=" * 72)

# --- Baseline (no shake) vs shake ---
print("\n-- Baseline (no shake) vs orbital shake --")
for label, shake_kw in [("no_shake", {}), ("orbital ", SHAKE)]:
    a1_v, h1_v, t_v = [], [], []
    for rep in range(2):
        t0 = time.time()
        results = await pr.read_absorbance(
            wavelength=600, wells=test_wells_16,
            use_new_return_type=True,
            **shake_kw,
        )
        elapsed = time.time() - t0
        t_v.append(elapsed)
        a1_v.append(results[0]["data"][0][0])
        h1_v.append(results[0]["data"][7][0])
        fmt16(results, f"{label} rep={rep+1} {elapsed:5.1f}s")
    summarize(a1_v, h1_v, t_v)

# --- Shake × ptpw ---
print("-- Shake × pause_time_per_well --")
for ptpw in [0, 5]:
    t0 = time.time()
    results = await pr.read_absorbance(
        wavelength=600, wells=test_wells_16,
        pause_time_per_well=ptpw,
        use_new_return_type=True,
        **SHAKE,
    )
    elapsed = time.time() - t0
    fmt16(results, f"shake + ptpw={ptpw:2d} {elapsed:5.1f}s")
print()

# --- Shake × stbm ---
print("-- Shake × settling_time_before_measurement --")
for stbm in [0, 3]:
    t0 = time.time()
    results = await pr.read_absorbance(
        wavelength=600, wells=test_wells_16,
        settling_time_before_measurement=stbm,
        use_new_return_type=True,
        **SHAKE,
    )
    elapsed = time.time() - t0
    fmt16(results, f"shake + stbm={stbm:2d} {elapsed:5.1f}s")
print()

# --- Shake × vertical ---
print("-- Shake × vertical --")
for vert in [False, True]:
    t0 = time.time()
    results = await pr.read_absorbance(
        wavelength=600, wells=test_wells_16,
        vertical=vert,
        use_new_return_type=True,
        **SHAKE,
    )
    elapsed = time.time() - t0
    fmt16(results, f"shake + vertical={str(vert):5s} {elapsed:5.1f}s")
print()

# --- Shake × ptpw × vertical (full combo) ---
print("-- Shake × ptpw=5 × vertical=True (full factorial) --")
t0 = time.time()
results = await pr.read_absorbance(
    wavelength=600, wells=test_wells_16,
    pause_time_per_well=5, vertical=True,
    use_new_return_type=True,
    **SHAKE,
)
elapsed = time.time() - t0
fmt16(results, f"shake + ptpw=5 + vert=True {elapsed:5.1f}s")
print()

# --- Shake type comparison ---
print("=" * 72)
print("Shake type comparison: 200 RPM × 7 s")
print("=" * 72)
for stype, slabel in [
    (ShakerType.ORBITAL, "orbital       "),
    (ShakerType.LINEAR, "linear        "),
    (ShakerType.DOUBLE_ORBITAL, "double_orbital"),
]:
    t0 = time.time()
    results = await pr.read_absorbance(
        wavelength=600, wells=test_wells_16,
        shake_type=stype, shake_speed_rpm=200, shake_duration_s=7,
        use_new_return_type=True,
    )
    elapsed = time.time() - t0
    fmt16(results, f"{slabel} {elapsed:5.1f}s")

In [ ]:
await pr.stop()

### Absorbance as transmittance

In [ ]:
results = await pr.read_absorbance(
    wavelength=450,
    report="transmittance",
    use_new_return_type=True
s)

# Transmittance is in percent (0-100)
print(f"Transmittance at 450nm, well A1: {results[0]['data'][0][0]}%")

### Multi-wavelength absorbance

Read up to 8 wavelengths in a single run. This is faster than running separate measurements because the plate is only scanned once.

Pass `wavelengths` as a backend keyword argument (this overrides the `wavelength` parameter):

In [ ]:
results = await pr.read_absorbance(
    wavelength=600,             # ignored when wavelengths is provided
    wavelengths=[260, 280, 450, 600, 750],
    use_new_return_type=True
)

# One result dict per wavelength
for r in results:
    print(f"Wavelength {r['wavelength']} nm -> A1 OD: {r['data'][0][0]}")

---
## Usage: Measuring Fluorescence

Fluorescence measures light emitted by a fluorophore after excitation at a specific wavelength.

### Optical paths

The CLARIOstar has three optical systems on the same mechanical rail. PyLabRobot currently uses the **monochromator path** for fluorescence. Here is how the three systems compare:

| Optical System | Spectral Range | Bandwidth | Used By PLR | Best For |
|----------------|---------------|-----------|:-----------:|----------|
| **Dual LVF Monochromator** | 320–840 nm | 8–100 nm (software selectable) | Yes | General fluorescence (GFP, mCherry, DAPI, etc.) |
| **Physical Filters** (11 slots) | 240–900 nm | Fixed per filter | Not yet | FP, HTRF, AlphaScreen, TR-FRET |
| **UV/Vis Spectrometer** | 220–1000 nm | 3 nm fixed | Yes (absorbance) | Absorbance only |

The monochromator offers freely tunable wavelengths and bandwidths, which is sufficient for most fluorescence assays. Physical filter slots (up to 4 excitation, 3 dichroic, 4 emission) provide higher sensitivity for specialized applications — see [Planned Features](#planned-features) for filter-based fluorescence support.

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `excitation_wavelength` | `int` | *required* | Excitation center wavelength in nm |
| `emission_wavelength` | `int` | *required* | Emission center wavelength in nm |
| `focal_height` | `float` | *required* | Focal height in mm (0–25) |
| `gain` | `int` | `1000` | Detector gain |
| `ex_bandwidth` | `int` | `20` | Excitation bandwidth in nm |
| `em_bandwidth` | `int` | `40` | Emission bandwidth in nm |
| `dichroic` | `int` | auto | Dichroic wavelength × 10 (auto-calculated as `(ex + em) * 5`) |
| `flashes` | `int` | `100` | Number of flashes per well (0–200) |
| `pause_time_per_well` | `int` | `0` | Per-well pause in deciseconds (0–10) |
| `settling_time_before_measurement` | `int` | `0` | Once-per-run post-shake settling delay in seconds |
| `bottom_optic` | `bool` | `False` | Read from bottom instead of top |

### Basic fluorescence read (e.g. GFP: ex 485 / em 528)

In [ ]:
results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    use_new_return_type=True
)

# results is a list with one dict containing:
#   "ex_wavelength": int, "em_wavelength": int,
#   "data": 8x12 grid, "temperature": float, "time": float
print(f"GFP signal at A1: {results[0]['data'][0][0]}")
print(f"Temperature: {results[0]['temperature']} \u00b0C")

### Fluorescence with custom gain, bandwidth, and flashes

In [ ]:
results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    gain=1500,           # increase gain for weak signals
    ex_bandwidth=10,     # narrow excitation bandwidth for better specificity
    em_bandwidth=20,     # narrow emission bandwidth
    flashes=150,         # more flashes for better signal-to-noise
    use_new_return_type=True
)

### Bottom-optic fluorescence

Use bottom-optic reading when working with cell monolayers, adherent cells, or clear-bottom plates where top-reading would be affected by the meniscus or lid:

In [ ]:
results = await pr.read_fluorescence(
    excitation_wavelength=544,
    emission_wavelength=590,
    focal_height=4.5,
    bottom_optic=True,
    use_new_return_type=True
)

---
## Usage: Measuring Luminescence

Luminescence measures light emitted by a chemical or biological reaction (no excitation light source).

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `focal_height` | `float` | `13` | Focal height in mm (0–25) |

### Basic luminescence read (full plate)

In [9]:
results = await pr.read_luminescence(
    focal_height=13.0,
    use_new_return_type=True
)

# results is a list of dicts, each containing:
#   "data": 8x12 grid of float values (rows x columns)
#   "temperature": float (Celsius)
#   "time": float (unix timestamp)
print(f"Temperature: {results[0]['temperature']} \u00b0C")
print(f"Well A1 value: {results[0]['data'][0][0]}")

Temperature: 26.0 °C
Well A1 value: 3891.5845732141615


---
## Usage: Partial Well Selection

All three measurement modes (luminescence, absorbance, fluorescence) support reading a subset of wells instead of the full plate. This can significantly speed up reads when you only need data from specific wells.

Unread wells are filled with `None` in the output grid.

In [ ]:
# Read only column 1 (wells A1-H1)
column_1_wells = [plate.get_well(f"{row}1") for row in "ABCDEFGH"]

results = await pr.read_absorbance(
    wavelength=600,
    wells=column_1_wells,
    use_new_return_type=True
)

# Only column 1 has values; all other wells are None
for row_idx, row_letter in enumerate("ABCDEFGH"):
    val = results[0]['data'][row_idx][0]  # column 0 = column 1
    print(f"Well {row_letter}1: OD = {val}")

In [ ]:
# Read specific wells by name
selected_wells = [plate.get_well("A1"), plate.get_well("D6"), plate.get_well("H12")]

results = await pr.read_luminescence(
    focal_height=13.0,
    wells=selected_wells,
    use_new_return_type=True
)

In [ ]:
# Partial well selection works for fluorescence too
row_A_wells = [plate.get_well(f"A{col}") for col in range(1, 13)]

results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    wells=row_A_wells,
    use_new_return_type=True
)

---
## Usage: Scan Mode Configuration

The scan mode controls how the plate reader traverses the plate during a measurement. Configurable via backend keyword arguments on all three read methods.

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `start_corner` | `StartCorner` | `TOP_LEFT` | Which corner to begin reading from |
| `unidirectional` | `bool` | `False` (lum/fl), `True` (abs) | Read in one direction only (vs. bidirectional zigzag) |
| `vertical` | `bool` | `True` | Read columns instead of rows (column-major, matches 8-channel pipette usage) |
| `flying_mode` | `bool` | `False` | Continuous movement (fluorescence only, max 3 flashes) |

Available start corners:
- `StartCorner.TOP_LEFT` (default)
- `StartCorner.TOP_RIGHT`
- `StartCorner.BOTTOM_LEFT`
- `StartCorner.BOTTOM_RIGHT`

In [ ]:
from pylabrobot.plate_reading.bmg_labtech.clariostar_backend import StartCorner

In [ ]:
# Read from bottom-right corner, vertical scan, unidirectional
results = await pr.read_absorbance(
    wavelength=600,
    start_corner=StartCorner.BOTTOM_RIGHT,
    unidirectional=True,
    vertical=True,
    use_new_return_type=True
)

In [ ]:
# Flying mode for fast fluorescence reads (max 3 flashes)
results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    flying_mode=True,
    flashes=3,
    start_corner=StartCorner.TOP_LEFT,
    use_new_return_type=True
)

```{note}
Flying mode keeps the optics head moving continuously instead of stopping at each well. This is much faster but limits you to a maximum of 3 flashes per well. Only available for fluorescence.
```

---
## Usage: Pre-Measurement Shaking

The CLARIOstar has a built-in shaker. When configured as part of a measurement call, the plate is shaken **immediately before** the optics head begins reading. This is useful for resuspending cells or mixing reagents right before a read.

```{note}
Currently, shaking is only available as a step embedded in a measurement run. Standalone shaking (shake without measuring) is a planned feature — see [Planned Features](#planned-features) below.
```

### Parameters

| Parameter | Type | Default | Description |
|-----------|------|---------|-------------|
| `shake_type` | `ShakerType` | `ORBITAL` | Type of shaking motion |
| `shake_speed_rpm` | `int` | `0` | Speed in RPM (100–700, in steps of 100) |
| `shake_duration_s` | `int` | `0` | Duration in seconds (0 = no shaking) |

Available shaker types:

| Type | Description | Max RPM |
|------|-------------|--------:|
| `ShakerType.ORBITAL` | Circular orbit | 700 |
| `ShakerType.LINEAR` | Back and forth in a line | 700 |
| `ShakerType.DOUBLE_ORBITAL` | Figure-eight pattern | 700 |
| `ShakerType.MEANDER` | Meandering path | **300** |

In [ ]:
from pylabrobot.plate_reading.bmg_labtech.clariostar_backend import ShakerType

In [ ]:
# Orbital shake at 300 RPM for 5 seconds before reading luminescence
results = await pr.read_luminescence(
    focal_height=13.0,
    shake_type=ShakerType.ORBITAL,
    shake_speed_rpm=300,
    shake_duration_s=5,
    use_new_return_type=True
)

In [ ]:
# Linear shake at 500 RPM for 10 seconds before reading absorbance
results = await pr.read_absorbance(
    wavelength=600,
    shake_type=ShakerType.LINEAR,
    shake_speed_rpm=500,
    shake_duration_s=10,
    use_new_return_type=True
)

In [ ]:
# Double orbital shake before fluorescence
results = await pr.read_fluorescence(
    excitation_wavelength=485,
    emission_wavelength=528,
    focal_height=8.5,
    shake_type=ShakerType.DOUBLE_ORBITAL,
    shake_speed_rpm=400,
    shake_duration_s=3,
    use_new_return_type=True
)

```{warning}
The `MEANDER` shaker type is limited to a maximum of 300 RPM. Exceeding this will raise a `ValueError`.
```

---
## Usage: Combining Features

All features can be combined in a single read call. Here is an example that uses partial well selection, pre-measurement shaking, and a custom scan configuration together:

In [ ]:
# Read fluorescence from wells A1-A12 with orbital shake, bottom-right start, and custom gain
row_A = [plate.get_well(f"A{col}") for col in range(1, 13)]

results = await pr.read_fluorescence(
    excitation_wavelength=544,
    emission_wavelength=590,
    focal_height=6.0,
    wells=row_A,
    gain=2000,
    flashes=50,
    bottom_optic=True,
    shake_type=ShakerType.ORBITAL,
    shake_speed_rpm=300,
    shake_duration_s=5,
    start_corner=StartCorner.BOTTOM_RIGHT,
    unidirectional=True,
    use_new_return_type=True
)

# Print row A results
for col_idx in range(12):
    val = results[0]['data'][0][col_idx]
    print(f"A{col_idx+1}: {val}")

---
## Usage: Non-Blocking Reads

By default, every `read_*` call blocks until the measurement is complete. For long-running operations (e.g. a 384-well plate with multiple wavelengths can take 10–15 minutes), you can start the measurement without waiting, do other work, and collect the results later.

Pass `wait=False` to any read method. The call returns `None` immediately after the machine starts measuring. Later, call the corresponding `collect_*_measurement()` method on the backend to retrieve and parse the results.

Poll `request_machine_status()` and wait for the `unread_data` flag to become `True` before collecting — this indicates that the machine has finished measuring and the results are ready to read.

In [9]:
# Start a multi-wavelength absorbance read without waiting
await pr.read_absorbance(
    wavelength=600,
    wavelengths=[260, 280, 450, 600],
    wait=False,
    use_new_return_type=True
)
# Returns None immediately — the machine is now measuring in the background

In [ ]:
# Poll until measurement data is available, then collect results
import asyncio

status = await clariostar_backend.request_machine_status()
while not status["unread_data"]:
    print("Measuring...")
    await asyncio.sleep(1)
    status = await clariostar_backend.request_machine_status()

# Retrieve and parse the data
results = await clariostar_backend.collect_absorbance_measurement(
    plate=plate,
    wells=plate.get_all_items(),
    wavelengths=[260, 280, 450, 600],
)

for r in results:
    print(f"Wavelength {r['wavelength']} nm -> A1 OD: {r['data'][0][0]}")

The same pattern works for fluorescence and luminescence:

| Read method | Collect method |
|-------------|---------------|
| `read_absorbance(..., wait=False)` | `clariostar_backend.collect_absorbance_measurement(plate, wells, wavelengths)` |
| `read_fluorescence(..., wait=False)` | `clariostar_backend.collect_fluorescence_measurement(plate, wells, ex_wl, em_wl)` |
| `read_luminescence(..., wait=False)` | `clariostar_backend.collect_luminescence_measurement(plate, wells)` |

```{note}
`wait=False` requires `use_new_return_type=True`. The `collect_*` methods are called directly on the backend (`clariostar_backend`), not the `PlateReader` frontend.
```

---
(reference-how-absorbance-calculation-works)=
## Reference: How Absorbance Calculation Works

The machine does not measure absorbance directly. It measures **raw detector counts** and the backend converts them to transmittance and then to OD. Understanding this pipeline helps when debugging unexpected values or when you need access to the raw data.

#### Detector channels

The CLARIOstar has four detector channels that operate simultaneously during an absorbance measurement:

| Channel | What it measures |
|---------|-----------------|
| **Chromatic 1** | Sample detector — the primary measurement at the requested wavelength |
| **Chromatic 2** | Secondary detector channel |
| **Chromatic 3** | Tertiary detector channel |
| **Reference** | Reference detector — tracks lamp intensity fluctuations across the plate |

Each channel has a calibration pair: a **high** value (100% transmission, measured through air) and a **low** value (0% transmission / dark current baseline). The high values are used in the OD conversion; the low (dark) values are embedded in the response but **not used** — see below.

#### Raw values returned by the firmware

The firmware returns four groups of values per measurement run (one per detector channel), followed by calibration data:

| Group | Count | What it is |
|-------|-------|-----------|
| **Group 0: Chromatic 1** | wells × wavelengths | Sample detector counts |
| **Group 1: Chromatic 2** | wells | Secondary detector counts |
| **Group 2: Chromatic 3** | wells | Tertiary detector counts |
| **Group 3: Reference** | wells | Reference detector counts |
| **Calibration** | 4 × (hi, lo) | One pair per channel — raw detector counts, same scale as data |

#### The conversion

The backend computes transmittance using a **reference-corrected formula (no dark subtraction)**:

**Step 1 — Normalize the sample signal:**
```
signal = sample / chromat_hi
```
This maps the sample reading onto a 0–1 scale relative to the 100% transmission calibration (measured through air). The `chromat_hi` value already includes the dark baseline, so subtracting `chromat_lo` would double-count it.

**Step 2 — Normalize the reference** (correct for lamp variation):
```
ref_norm = ref_hi / ref_well
```
This corrects for per-well lamp intensity differences. If a flash was slightly brighter for one well, both sample and reference see the same increase, and dividing cancels it out.

**Step 3 — Calculate percent transmittance:**
```
T% = signal × ref_norm × 100 = (sample / chromat_hi) × (ref_hi / ref_well) × 100
```
An empty well gives T% ≈ 100%.

**Step 4 — Convert to OD** (log transform):
```
OD = -log10(T% / 100) = -log10(signal × ref_norm)
```
This is the Beer-Lambert conversion. A blank well (T% ≈ 100) gives OD ≈ 0. A dense bacterial culture (T% ≈ 10) gives OD ≈ 1.0.

| Transmittance | OD | Meaning |
|--------------:|---:|---------|
| 100% | 0.0 | No absorption (blank / empty well) |
| 10% | 1.0 | 90% of light absorbed |
| 1% | 2.0 | 99% of light absorbed |

```{note}
The `chromat_lo` and `ref_lo` dark calibration values are embedded in every measurement response but are **not used** in the OD calculation. The `chromat_hi` calibration already includes the dark baseline (it's a raw count, not a dark-subtracted count), so subtracting `chromat_lo` would double-count the dark current. This was verified by matching all 96 wells of a test plate against OEM MARS software output to within ±0.001 OD.
```

#### Report modes

| `report=` | What you get | Description |
|-----------|-------------|:------------|
| `"OD"` (default) | Optical density values | Reference-corrected, log-transformed |
| `"transmittance"` | Percent transmittance (0–100) | Reference-corrected |
| `"raw"` | Raw detector counts + calibration data | Unprocessed firmware values |

The `"raw"` report mode returns each wavelength dict with additional keys: `references` (per-well reference counts), `chromatic_cal` (hi/lo pair for that wavelength), `reference_cal` (hi/lo pair for the reference channel), and the secondary/tertiary channel data (`chromatic2`, `chromatic3`, `chromatic2_cal`, `chromatic3_cal`). This is useful for custom calibration pipelines, quality control, or investigating detector behavior.

---
(planned-features)=
## Planned Features

The CLARIOstar firmware supports several additional features that are documented in the BMG ActiveX/DDE manual but **not yet implemented** in the PyLabRobot backend. These require USB traffic captures of the corresponding firmware byte sequences.

### Filter-Based Fluorescence

The CLARIOstar has 11 physical filter slots (4 excitation, 3 dichroic, 4 emission) alongside the LVF monochromator, sharing the same mechanical rail. Physical filters provide higher sensitivity than the monochromator and are required for certain assay types:

| Assay Type | Why Filters Are Needed |
|------------|----------------------|
| **Fluorescence Polarization (FP)** | Requires polarizer filters in excitation and emission positions |
| **HTRF / TR-FRET** | Needs dedicated time-resolved filters + dual chromatics (e.g. Fura-2) |
| **AlphaScreen / AlphaLISA** | Uses dedicated laser + specific filter sets |
| **High-sensitivity FI** | Filters give ~2× better sensitivity (0.15 pM vs 0.35 pM Fluorescein at top) |

Sensitivity comparison (per operating manual, Fluorescein, 384sv, 20 µL):

| Path | Top Reading | Bottom Reading |
|------|-----------|---------------|
| Filters | 0.15 pM (< 3 amol/well) | 1.0 pM (< 50 amol/well) |
| Monochromator | 0.35 pM (< 7 amol/well) | 3.0 pM (< 150 amol/well) |

```python
# Planned API (not yet implemented)
results = await pr.read_fluorescence(
    excitation_wavelength=485,     # for metadata only in filter mode
    emission_wavelength=528,
    focal_height=8.5,
    optic_path="filter",           # switch from monochromator to filters
    ex_filter_position=1,          # excitation filter slot 1-4
    em_filter_position=5,          # emission filter slot 5-8
    dichroic_position="A",         # dichroic mirror slot A/B/C
    use_new_return_type=True,
)
```

Filter positions correspond to the physical slots visible when opening the filter cover (see Section 6.3 of the operating manual). To check which filters are installed, use the OEM control software: **Settings → Filter → Detect all filters** (password: `bmg`).

See `CLARIOSTAR_OPTICAL_PATH_PLAN.md` for the full implementation plan including USB capture steps.

### Spectral Scanning

The CLARIOstar's monochromator hardware supports continuous wavelength scanning, where the instrument sweeps through a range of wavelengths and returns an intensity value at each step. This is useful for identifying unknown absorption/emission peaks, verifying filter selection, or characterizing new fluorophores.

```python
# Planned API (not yet implemented)
spectrum = await clariostar_backend.absorbance_scan(
    plate=plate,
    wells=[plate.get_well("A1")],
    wavelength_start=220,
    wavelength_end=750,
    wavelength_step=2,
)
# spectrum["A1"] -> [(220, 0.05), (222, 0.06), ..., (750, 0.02)]
```

Both absorbance and fluorescence spectral scans are supported by the hardware:
- **Absorbance scan**: Sweeps the monochromator across a wavelength range, measuring OD at each step. Useful for finding the absorption peak of a dye or checking sample purity (e.g. A260/A280 ratio for nucleic acids).
- **Fluorescence excitation scan**: Fixes the emission wavelength and sweeps the excitation monochromator.
- **Fluorescence emission scan**: Fixes the excitation wavelength and sweeps the emission monochromator.

### Standalone Shaking

The firmware supports a dedicated `Shake` command independent of any measurement run. This would enable:

```python
# Planned API (not yet implemented)
await clariostar_backend.shake(
    shake_type=ShakerType.ORBITAL,
    speed_rpm=300,
    duration_s=60
)
```

Per the OEM manual, standalone shaking supports:
- 5 shake types: orbital, linear, double orbital, meander corner well, orbital corner well
- Speed: 100–700 RPM (up to 1100 with high-speed shaking option)
- Duration: 1–3600 seconds
- Optional X/Y position parameters

### Idle Movement

The firmware supports an `IdleMove` command for continuous plate movement between measurements (useful during long incubation periods):

```python
# Planned API (not yet implemented)
await clariostar_backend.idle_move(mode="orbital", speed_rpm=200, duration_s=300)
await clariostar_backend.idle_move(mode="cancel")  # stop idle movement
```

Per the OEM manual, 7 modes are available including linear corner-to-corner movement, incubation position waiting, and various shaking patterns with configurable on/off cycling.

### Injector System

The CLARIOstar supports up to two injector pumps (`Pump1` / `Pump2`) for dispensing reagents during kinetic reads. Not yet implemented.

```{note}
To help implement any of these features, capture USB traffic while executing the corresponding command in the BMG OEM control software and open an issue on the [PyLabRobot GitHub](https://github.com/PyLabRobot/pylabrobot/issues).
```

---
## Closing Connection

In [ ]:
pr.unassign_child_resource(plate)

In [ ]:
await pr.stop()

This closes the FTDI connection. After calling `stop()`, you must call `setup()` again before using the plate reader.